<h1><center> Python project</center></h1>
<h2><center>Current car prices and other relevant parameters from bazos.cz</center></h2>
<h3><center>Daniel Brosko, Vojtěch Suchánek</center></h3>

Our goal is to web-scrape advertisements listed on website bazos.cz, which is currently one of the most used websites for selling used cars in Czech republic. It has more than 15 000 car adds daily. On the other hand, it has really poor search options, which pretty much complicates searching for desired car based on your parameters.

We are going to code algorithm, which will scan adds for the current day, pick those, which fulfill our conditions on date and car type and save their links. Then we will go to each link and save the text of the add. Then we will try to analyze the text of the add to find our parameters. In the end, we will perform some visualizations of distribution of price and selected parameters and try to select some underprices offers.

This task might be better to perform continuouslly, for example each hour, to not overload the website. This approach might also allow for longer time period to analyze the data. But since this project should be designed as one-time run, we decided to limit the data to only current date.

In [56]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import date, datetime, timedelta
import time
import numpy as np
import pandas as pd

In [51]:
# Just to get the version of packages for requirements.txt
#pip list

In [18]:
bots = requests.get('https://auto.bazos.cz/robots.txt')
#print(bots.text)

From the robots page we can see that our actions done in our projects are allowed, since we are not gonna use these search commands.

Here we will get the main page from car section of bazos.

In [19]:
# firstly, we define the input variable so the user can search according to their preference
user_search_input = str("octávia 3")
user_search_input = re.sub(r"[^\w\s]", '', user_search_input)
user_search_input = re.sub(r"\s+", '+', user_search_input)

no_of_adv_url = 'https://auto.bazos.cz/0/?hledat=' + user_search_input + '&hlokalita=&humkreis=25&cenaod=&cenado=&order='

page = requests.get(no_of_adv_url)
no_of_adv_html = BeautifulSoup(page.text, 'html')

get_no_adv = no_of_adv_html.find('div', {'class':'listainzerat inzeratyflex'})
get_no_adv = get_no_adv.find('div', {'class':'inzeratynadpis'})
get_no_adv = get_no_adv.text
get_no_adv = get_no_adv.split("z ")[-1]
number_adv = get_no_adv.replace(" ","")
number_adv = int(number_adv)
# works properly, "number_adv" represents total number of advertisements for particular search input,
# however, we modify it so that it correspondents correctly to the structure of page
no_of_ad_lists = number_adv // 20
if number_adv % 20 == 0:
    number_adv = no_of_ad_lists * 19
else:
    number_adv = no_of_ad_lists * 20


number_sequence = range(0, number_adv, 20) #start, stop (not included), step
# we create empty list for saving the urls,
# and then append other tabs with offers (since there are only 20 offers per tab by default)
main_url_list = list()

for i in range(0,len(number_sequence)):
    url = f'https://auto.bazos.cz/{number_sequence[i]}/'
    url = url + '?hledat=' + user_search_input + '&hlokalita=&humkreis=25&cenaod=&cenado=&order='
    main_url_list.append(url)
# here we can check the list of urls for particular tabs
#print(main_url_list)

['https://auto.bazos.cz/0/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/20/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/40/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/60/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/80/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/100/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/120/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/140/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/160/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/180/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/200/?hledat=octávia+3&hlokalita=&humkreis=25&cenaod=

In [33]:
# in the next step, we get the text of each of those tabs using the BeautifulSoup function,
# and save it as elements of the "soup_list"
soup_list = list()
for url in main_url_list:
    page = requests.get(url)

    ## MAYBE SLOW DOWN LATER by 0.3s per iteration
    soup_list.append(BeautifulSoup(page.text, 'html'))

# filter to cut-off pseudo-empty elements in main_url_list to prevent the unwanted behaviour of the code
res_soup_list = []
for element in soup_list:
    if "html" in element:
        res_soup_list.append(element)
soup_list = res_soup_list

There are 20 adds plus other page elements. Lets pick just one add to see their construction.

There are several elements, which are important to us. In this section, we are interested in add id, which we are able to extract from href in element a on the second line. Its the number after "inzerat/". We are also interested in the href itself, since it is our link to follow to the actual add. Another element important to us is h2 class = "nadpis", which contains title of the add, from which we can extract type of car. And the last interesting element is span with class "velikost10", which reffers to date uploaded.

Now we can follow the link to the actual add page to see its content.

In the next chunk, we filter for the ads added today - if we compare the numbers, now we have today's 98 of out entire 2020 (as of now) avertisements.

In [80]:
#DODELANY FILTER NA POCET DNU ZPATKY
number_of_accepted_past_days = 5 #range (0:5)
today = date.today()
accepted_days = list()

for i in list(range(0, min(number_of_accepted_past_days + 1, 6), 1)):
    i_days_ago = today - timedelta(days=i)
    accepted_days.append(i_days_ago)
    
# by the following code, we get the urls of each advertisement /offer/ (listed in the tabs we work with),
# and save it to "list_of_offers_url"
list_of_offers_url = list()

for element in soup_list:
    x = element.find_all('div', {'class':'inzeraty inzeratyflex'})
    
    for sub_element in x:
        y = sub_element.find('div', {'class':'inzeratynadpis'})
        attribute_a = y.find('a')
        w = attribute_a.get('href')
        attribute_span = y.find('span')
        
        #here we just obtain the date from the particular advertisement to make sure we analyze only ads added today
        date_str = str(attribute_span.text)
        date_str = date_str.replace(" ","")
        date_str = date_str.strip("-[]")
        date_str = date_str.strip("TOP-[]")
        date_object = datetime.strptime(date_str,'%d.%m.%Y')
        date_object = date_object.date()

        if not date_object in accepted_days:
     #   if date_object != today:
            continue
    
        list_of_offers_url.append(f'https://auto.bazos.cz{w}')
    
print(len(list_of_offers_url))
print(list_of_offers_url)

473
['https://auto.bazos.cz/inzerat/157075143/skoda-octavia-3-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/157035970/octavia-scout-4x420tdi-110kwr2018dsgledacc123tiskm.php', 'https://auto.bazos.cz/inzerat/157035183/octavia-rs-20tsi-230psm2018facecantoncolumbuslane.php', 'https://auto.bazos.cz/inzerat/156861400/skoda-octavia-3-20tdi-110kw-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156839898/skoda-octavia-3-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156800462/skoda-octavia-3-16tdi-85kw-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156776866/skoda-octavia-3-fc-14tsi-cng-81kw-dsg-style-koupcr107tkm.php', 'https://auto.bazos.cz/inzerat/156583245/skoda-octavia-3-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/157066972/skoda-octavia-rs-dsg-fullled-acc-columbus-vyhrsedacky.php', 'https://auto.bazos.cz/inzerat/157066943/skoda-octavia-iii-rs-fullled-navi-panor

In [36]:
today = date.today()
# by the following code, we get the urls of each advertisement /offer/ (listed in the tabs we work with),
# and save it to "list_of_offers_url"
list_of_offers_url = list()

for element in soup_list:
    x = element.find_all('div', {'class':'inzeraty inzeratyflex'})
    
    for sub_element in x:
        y = sub_element.find('div', {'class':'inzeratynadpis'})
        attribute_a = y.find('a')
        w = attribute_a.get('href')
        attribute_span = y.find('span')
        
        #here we just obtain the date from the particular advertisement to make sure we analyze only ads added today
        date_str = str(attribute_span.text)
        date_str = date_str.replace(" ","")
        date_str = date_str.strip("-[]")
        date_str = date_str.strip("TOP-[]")
        date_object = datetime.strptime(date_str,'%d.%m.%Y')
        date_object = date_object.date()

        if date_object != today:
            continue
    
        list_of_offers_url.append(f'https://auto.bazos.cz{w}')
        #    time.sleep(0.03)
    
print(list_of_offers_url)

['https://auto.bazos.cz/inzerat/157075143/skoda-octavia-3-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/157035970/octavia-scout-4x420tdi-110kwr2018dsgledacc123tiskm.php', 'https://auto.bazos.cz/inzerat/157035183/octavia-rs-20tsi-230psm2018facecantoncolumbuslane.php', 'https://auto.bazos.cz/inzerat/156861400/skoda-octavia-3-20tdi-110kw-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156839898/skoda-octavia-3-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156800462/skoda-octavia-3-16tdi-85kw-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156776866/skoda-octavia-3-fc-14tsi-cng-81kw-dsg-style-koupcr107tkm.php', 'https://auto.bazos.cz/inzerat/156583245/skoda-octavia-3-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/157066972/skoda-octavia-rs-dsg-fullled-acc-columbus-vyhrsedacky.php', 'https://auto.bazos.cz/inzerat/157066943/skoda-octavia-iii-rs-fullled-navi-panorama-

In [52]:
# DATA/TEXT MINING PART
def remove_spaces(text):
    return text.replace(" ", "")
def get_numbers_from_text(text):
    import re
    text = remove_spaces(text)
    pattern = '[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?'
    list_of_numbers = re.findall(pattern, text)
    return list_of_numbers
def find_years(numbers):
    numbers = [x for x in numbers if (float(x) > 1980) and (float(x) < 2030)]
    return numbers
def find_km(numbers):
    numbers = [x for x in numbers if (float(x) > 3000) and (float(x) < 500000)]
    return numbers
def get_context(text, list_of_tokens, year_dictionary = ['egistr', 'r.v', 'vyrob', 'výrob', 'prov'], km_dictionary = ['km', 'Km', 'KM', 'ilomet', 'ajet', 'ájez', 'achom', 'atoč'], context_span=20):
    import re
    context = []
    year = 'No match'
    km = 'No match'
    for token in find_years(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in year_dictionary:
                year_find = [m.start() for m in re.finditer(s, substring)]
                if len(year_find) > 0:
                    year = token
    for token in find_km(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in km_dictionary:
                km_find = [m.start() for m in re.finditer(s, substring)]
                if len(km_find) > 0:
                    km = token
    return [year, km]
def get_info(links):
    results = []
    for i in links:
        add_page = requests.get(i)
        soup_add = BeautifulSoup(add_page.text, 'html')
        add = remove_spaces(soup_add.find('div', {'class':'popisdetail'}).get_text())
        all_numbers = get_numbers_from_text(add)
        context_got = get_context(add, all_numbers)
        result = [i, context_got[0], context_got[1]]
        results.append(result)
        time.sleep(0.03)
    return results
link = ['https://auto.bazos.cz/inzerat/157012769/vw-golf-7-20tdi-110kw-107tkm-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156954743/skoda-octavia-3dsg-20-tdi.php']
#link = list_of_offers_url
get_info(link)

AttributeError: 'NoneType' object has no attribute 'get_text'

Here we can see the description, which we will use to extract parameters of the car. This will be one of the most difficult parts of the project to extract the relevant text in unformated text. There is no official format to the text, so we will have to find a way how to extract this information from various formats.

After retrieving these informations, we will use analysis of the data got from this process.